# Map Layer Groups

In [ ]:
import os
import time

from felt_python import (
    create_map,
    delete_map,
    list_layer_groups,
    get_layer_group,
    update_layer_groups,
    delete_layer_group,
    publish_layer_group,
    update_layers,
    get_layer,
    upload_file
)

os.environ["FELT_API_TOKEN"] = "<YOUR_API_TOKEN>"

# Create a map for testing

In [ ]:
resp = create_map(
    title="A felt-python map for testing layer groups",
    lat=40,
    lon=-3,
    zoom=5,
    public_access="private",
)
map_id = resp["id"]
resp["url"]

# Upload some layers to work with

We'll upload a few layers to organize into groups

In [ ]:
# Upload first layer
layer1_resp = upload_file(
    map_id=map_id,
    file_name="fixtures/null-island-points-sample.geojson",
    layer_name="Points Layer"
)
layer1_id = layer1_resp["layer_id"]

# Upload second layer
layer2_resp = upload_file(
    map_id=map_id,
    file_name="fixtures/null-island-polygons-wkt.csv",
    layer_name="Polygons Layer"
)
layer2_id = layer2_resp["layer_id"]

# Wait for processing to complete
while get_layer(map_id, layer1_id)["progress"] < 100:
    print("Waiting for layer to finish processing...")
    time.sleep(5)while get_layer(map_id, layer2_id)["progress"] < 100:
    print("Waiting for layer to finish processing...")
    time.sleep(5)print("Layers ready...")


# List existing layer groups

Initially, there should be no layer groups

In [ ]:
initial_groups = list_layer_groups(map_id)
initial_groups

# Create layer groups

Create groups and assign layers to them

In [ ]:
# Create two layer groups
layer_groups = [
    {
        "name": "Vector Data",
        "caption": "A collection of vector datasets"
    },
    {
        "name": "Base Data",
        "caption": "Reference layers"
    }
]

created_groups = update_layer_groups(map_id, layer_groups)
created_groups

# Retrieve a specific layer group's details

In [ ]:
# Get details for the first group
group_id = created_groups[0]["id"]
group_details = get_layer_group(map_id, group_id)
group_details

# Update layer groups

Update the groups to change properties

In [ ]:
# Retrieve current groups
current_groups = list_layer_groups(map_id)
group1_id = current_groups[0]["id"]
group2_id = current_groups[1]["id"]

# Update the groups
updated_groups = [
    {
        "id": group1_id,
        "name": "Vector Data (Updated)",
        "caption": "A collection of vector datasets (updated)",
        "ordering_key": 1
    },
    {
        "id": group2_id,
        "name": "Base Data (Updated)",
        "caption": "Reference layers (updated)",
        "ordering_key": 2
    }
]

result = update_layer_groups(map_id, updated_groups)
result

# Update layers to assign them to groups


In [ ]:
# Prepare updates for both layers
layer_updates = [
    {
        "id": layer1_id
        "layer_group_id": group1_id,
    },
    {
        "id": layer2_id
        "layer_group_id": group2_id,
    }
]

updated_layers = update_layers(map_id, layer_updates)

# Publish a layer group to the library

In [ ]:
# Publish the first group
published_group = publish_layer_group(
    map_id=map_id,
    layer_group_id=group1_id,
    name="Published Vector Data"
)
published_group

# Delete a layer group

In [ ]:
# Delete the second group
delete_layer_group(map_id, group2_id)

# Verify deletion
remaining_groups = list_layer_groups(map_id)
print(f"Groups remaining: {len(remaining_groups)}")
remaining_groups

# Clean up

In [ ]:
delete_map(map_id)